In [ ]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from datetime import datetime
import json

KAFKA_TOPIC = "uk-flood"
BOOTSTRAP_SERVER = "localhost:9092"

ssc = StreamingContext(sc,1) #stream each one second
ssc.checkpoint("./checkpoint")
lines = KafkaUtils.createDirectStream(ssc, [KAFKA_TOPIC],
                                      {"metadata.broker.list": BOOTSTRAP_SERVER})

# Function to calculate the severity of each area
def calculate_severity_each_area(lines, window_length = 10, sliding_interval = 10):
    def compute_severity_each_area(rdd):
        items = rdd.collect()
        if len(items) > 0:
            severity_each_area = {}
            for item in items:
                if not(item['eaAreaName'] in severity_each_area):
                    severity_each_area[item['eaAreaName']] = [
                        {
                            'severityLevel': 1,
                            'count': 0
                        },
                        {
                            'severityLevel': 2,
                            'count': 0
                        },
                        {
                            'severityLevel': 3,
                            'count': 0
                        },
                        {
                            'severityLevel': 4,
                            'count': 0
                        }
                    ]
                
                severity_each_area[item['eaAreaName']][item['severityLevel'] - 1]['count'] += 1

            result = []
            for area in severity_each_area:
                for severity in severity_each_area[area]:
                    result.append({"flood_warning_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "areaName": area, "severityLevel": severity['severityLevel'], "count": severity['count']})
        else:
            result = []
        return rdd.context.parallelize(result)  # Convert the list to an RDD
    
    data = lines.map(lambda line: json.loads(line[1])).flatMap(lambda items: items)
    windowed_data = data.window(window_length, sliding_interval)
    result = windowed_data.transform(compute_severity_each_area)

    return result

# run the function
result = calculate_severity_each_area(lines, window_length=10, sliding_interval=10)
# Print
result.pprint()
ssc.start()
ssc.awaitTermination()